In [1]:
import numpy as np
import pandas as pd
import lightgbm as gbm


In [2]:
train = pd.read_csv("../input/application_train.csv") 
test = pd.read_csv('../input/application_test.csv')
bureau = pd.read_csv('../input/bureau.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
credit_card_balance = pd.read_csv('../input/credit_card_balance.csv')
pos_cash_balance = pd.read_csv('../input/POS_CASH_balance.csv')
previous_applications = pd.read_csv('../input/previous_application.csv')

    

In [3]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [5]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [6]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [7]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [8]:
previous_applications.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
from sklearn.preprocessing import LabelEncoder

def encoding_missing_values(df):
    label_encoder = LabelEncoder()
    categorical_variables = df.select_dtypes('object').columns.tolist()
    for column in categorical_variables:
        df[column] = label_encoder.fit_transform(df[column].astype(str))
    numerical_variables=df.select_dtypes(['float64','int64']).columns.tolist()
    df[numerical_variables]= df[numerical_variables].fillna(0)
    return df
    

In [10]:
train_shape = train.shape[0]
target = train.TARGET.copy()
train = train.drop(labels = ['TARGET'], axis = 1)

train = encoding_missing_values(train)
test = encoding_missing_values(test)
bureau = encoding_missing_values(bureau)
bureau_balance = encoding_missing_values(bureau_balance)
credit_card_balance = encoding_missing_values(credit_card_balance)
pos_cash_balance = encoding_missing_values(pos_cash_balance)
previous_applications = encoding_missing_values(previous_applications)

In [11]:
bureau_labels=[n+'_'+l for n in bureau.columns.tolist() if n!='SK_ID_CURR' for l in ['mean','count','median','max']]
bureau_stats=bureau.groupby('SK_ID_CURR').agg(['mean','count','median','max']).reset_index()
bureau_stats.columns=['SK_ID_CURR']+bureau_labels
bureau_stats.head()

,SK_ID_CURR,SK_ID_BUREAU_mean,SK_ID_BUREAU_count,SK_ID_BUREAU_median,SK_ID_BUREAU_max,CREDIT_ACTIVE_mean,CREDIT_ACTIVE_count,CREDIT_ACTIVE_median,CREDIT_ACTIVE_max,CREDIT_CURRENCY_mean,CREDIT_CURRENCY_count,CREDIT_CURRENCY_median,CREDIT_CURRENCY_max,DAYS_CREDIT_mean,DAYS_CREDIT_count,DAYS_CREDIT_median,DAYS_CREDIT_max,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_median,CREDIT_DAY_OVERDUE_max,DAYS_CREDIT_ENDDATE_mean,DAYS_CREDIT_ENDDATE_count,DAYS_CREDIT_ENDDATE_median,DAYS_CREDIT_ENDDATE_max,DAYS_ENDDATE_FACT_mean,DAYS_ENDDATE_FACT_count,DAYS_ENDDATE_FACT_median,DAYS_ENDDATE_FACT_max,AMT_CREDIT_MAX_OVERDUE_mean,AMT_CREDIT_MAX_OVERDUE_count,AMT_CREDIT_MAX_OVERDUE_median,AMT_CREDIT_MAX_OVERDUE_max,CNT_CREDIT_PROLONG_mean,CNT_CREDIT_PROLONG_count,CNT_CREDIT_PROLONG_median,CNT_CREDIT_PROLONG_max,AMT_CREDIT_SUM_mean,AMT_CREDIT_SUM_count,AMT_CREDIT_SUM_median,AMT_CREDIT_SUM_max,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_median,AMT_CREDIT_SUM_DEBT_max,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_LIMIT_count,AMT_CREDIT_SUM_LIMIT_median,AMT_CREDIT_SUM_LIMIT_max,AMT_CREDIT_SUM_OVERDUE_mean,AMT_CREDIT_SUM_OVERDUE_count,AMT_CREDIT_SUM_OVERDUE_median,AMT_CREDIT_SUM_OVERDUE_max,CREDIT_TYPE_mean,CREDIT_TYPE_count,CREDIT_TYPE_median,CREDIT_TYPE_max,DAYS_CREDIT_UPDATE_mean,DAYS_CREDIT_UPDATE_count,DAYS_CREDIT_UPDATE_median,DAYS_CREDIT_UPDATE_max,AMT_ANNUITY_mean,AMT_ANNUITY_count,AMT_ANNUITY_median,AMT_ANNUITY_max
0,100001,5896633.000,7,5896633.0,5896636,1.142857,7,2.0,2,0.0,7,0.0,0,-735.000000,7,-857.0,-49,0.0,7,0.0,0,82.428571,7,-179.0,1778.0,-471.714286,7,-544.0,0.0,0.000000,7,0.0,0.000,0.0,7,0.0,0,207623.571429,7,168345.00,378000.0,85240.928571,7,0.0,373239.0,0.000000,7,0.0,0.000,0.0,7,0.0,0.0,3.000000,7,3.0,3,-93.142857,7,-155.0,-6,3545.357143,7,0.0,10822.5
1,100002,6153272.125,8,6158905.5,6158909,1.500000,8,2.0,2,0.0,8,0.0,0,-874.000000,8,-1042.5,-103,0.0,8,0.0,0,-261.750000,8,0.0,780.0,-523.125000,8,-479.5,0.0,1050.643125,8,0.0,5043.645,0.0,8,0.0,0,108131.945625,8,54130.50,450000.0,30722.625000,8,0.0,245781.0,3998.570625,8,0.0,31988.565,0.0,8,0.0,0.0,3.500000,8,3.5,4,-499.875000,8,-402.5,-7,0.000000,8,0.0,0.0
2,100003,5885878.500,4,5885878.5,5885880,1.500000,4,2.0,2,0.0,4,0.0,0,-1400.750000,4,-1205.5,-606,0.0,4,0.0,0,-544.500000,4,-480.0,1216.0,-823.000000,4,-580.5,0.0,0.000000,4,0.0,0.000,0.0,4,0.0,0,254350.125000,4,92576.25,810000.0,0.000000,4,0.0,0.0,202500.000000,4,0.0,810000.000,0.0,4,0.0,0.0,3.500000,4,3.5,4,-816.000000,4,-545.0,-43,0.000000,4,0.0,0.0
3,100004,6829133.500,2,6829133.5,6829134,2.000000,2,2.0,2,0.0,2,0.0,0,-867.000000,2,-867.0,-408,0.0,2,0.0,0,-488.500000,2,-488.5,-382.0,-532.500000,2,-532.5,-382.0,0.000000,2,0.0,0.000,0.0,2,0.0,0,94518.900000,2,94518.90,94537.8,0.000000,2,0.0,0.0,0.000000,2,0.0,0.000,0.0,2,0.0,0.0,3.000000,2,3.0,3,-532.000000,2,-532.0,-382,0.000000,2,0.0,0.0
4,100005,6735201.000,3,6735201.0,6735202,0.666667,3,0.0,2,0.0,3,0.0,0,-190.666667,3,-137.0,-62,0.0,3,0.0,0,439.333333,3,122.0,1324.0,-41.000000,3,0.0,0.0,0.000000,3,0.0,0.000,0.0,3,0.0,0,219042.000000,3,58500.00,568800.0,189469.500000,3,25321.5,543087.0,0.000000,3,0.0,0.000,0.0,3,0.0,0.0,3.333333,3,3.0,4,-54.333333,3,-31.0,-11,1420.500000,3,0.0,4261.5


In [12]:
credit_card_balance_labels=[n+'_'+l for n in credit_card_balance.columns.tolist() if n!='SK_ID_CURR' for l in ['mean','count','median','max']]
credit_card_stats=credit_card_balance.groupby('SK_ID_CURR').agg(['mean','count','median','max']).reset_index()
credit_card_stats.columns=['SK_ID_CURR']+credit_card_balance_labels
credit_card_stats.head()

,SK_ID_CURR,SK_ID_PREV_mean,SK_ID_PREV_count,SK_ID_PREV_median,SK_ID_PREV_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_count,MONTHS_BALANCE_median,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_count,AMT_BALANCE_median,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,AMT_CREDIT_LIMIT_ACTUAL_count,AMT_CREDIT_LIMIT_ACTUAL_median,AMT_CREDIT_LIMIT_ACTUAL_max,AMT_DRAWINGS_ATM_CURRENT_mean,AMT_DRAWINGS_ATM_CURRENT_count,AMT_DRAWINGS_ATM_CURRENT_median,AMT_DRAWINGS_ATM_CURRENT_max,AMT_DRAWINGS_CURRENT_mean,AMT_DRAWINGS_CURRENT_count,AMT_DRAWINGS_CURRENT_median,AMT_DRAWINGS_CURRENT_max,AMT_DRAWINGS_OTHER_CURRENT_mean,AMT_DRAWINGS_OTHER_CURRENT_count,AMT_DRAWINGS_OTHER_CURRENT_median,AMT_DRAWINGS_OTHER_CURRENT_max,AMT_DRAWINGS_POS_CURRENT_mean,AMT_DRAWINGS_POS_CURRENT_count,AMT_DRAWINGS_POS_CURRENT_median,AMT_DRAWINGS_POS_CURRENT_max,AMT_INST_MIN_REGULARITY_mean,AMT_INST_MIN_REGULARITY_count,AMT_INST_MIN_REGULARITY_median,AMT_INST_MIN_REGULARITY_max,AMT_PAYMENT_CURRENT_mean,AMT_PAYMENT_CURRENT_count,AMT_PAYMENT_CURRENT_median,...,AMT_RECIVABLE_mean,AMT_RECIVABLE_count,AMT_RECIVABLE_median,AMT_RECIVABLE_max,AMT_TOTAL_RECEIVABLE_mean,AMT_TOTAL_RECEIVABLE_count,AMT_TOTAL_RECEIVABLE_median,AMT_TOTAL_RECEIVABLE_max,CNT_DRAWINGS_ATM_CURRENT_mean,CNT_DRAWINGS_ATM_CURRENT_count,CNT_DRAWINGS_ATM_CURRENT_median,CNT_DRAWINGS_ATM_CURRENT_max,CNT_DRAWINGS_CURRENT_mean,CNT_DRAWINGS_CURRENT_count,CNT_DRAWINGS_CURRENT_median,CNT_DRAWINGS_CURRENT_max,CNT_DRAWINGS_OTHER_CURRENT_mean,CNT_DRAWINGS_OTHER_CURRENT_count,CNT_DRAWINGS_OTHER_CURRENT_median,CNT_DRAWINGS_OTHER_CURRENT_max,CNT_DRAWINGS_POS_CURRENT_mean,CNT_DRAWINGS_POS_CURRENT_count,CNT_DRAWINGS_POS_CURRENT_median,CNT_DRAWINGS_POS_CURRENT_max,CNT_INSTALMENT_MATURE_CUM_mean,CNT_INSTALMENT_MATURE_CUM_count,CNT_INSTALMENT_MATURE_CUM_median,CNT_INSTALMENT_MATURE_CUM_max,NAME_CONTRACT_STATUS_mean,NAME_CONTRACT_STATUS_count,NAME_CONTRACT_STATUS_median,NAME_CONTRACT_STATUS_max,SK_DPD_mean,SK_DPD_count,SK_DPD_median,SK_DPD_max,SK_DPD_DEF_mean,SK_DPD_DEF_count,SK_DPD_DEF_median,SK_DPD_DEF_max
0,100006,1489396.0,6,1489396.0,1489396,-3.5,6,-3.5,-1,0.000000,6,0.0,0.00,270000.000000,6,270000,270000,0.000000,6,0.0,0.0,0.000000,6,0.0,0.0,0.0,6,0.0,0.0,0.0,6,0.0,0.0,0.000000,6,0.0,0.0,0.000000,6,0.000,...,0.000000,6,0.0,0.00,0.000000,6,0.0,0.00,0.000000,6,0.0,0.0,0.000000,6,0.0,0,0.0,6,0.0,0.0,0.0,6,0.0,0.0,0.000000,6,0.0,0.0,0.000000,6,0,0,0.000000,6,0.0,0,0.000000,6,0.0,0
1,100011,1843384.0,74,1843384.0,1843384,-38.5,74,-38.5,-2,54482.111149,74,0.0,189000.00,164189.189189,74,180000,180000,2432.432432,74,0.0,180000.0,2432.432432,74,0.0,180000.0,0.0,74,0.0,0.0,0.0,74,0.0,0.0,3902.759392,74,0.0,9000.0,4843.064189,74,563.355,...,54433.179122,74,0.0,189000.00,54433.179122,74,0.0,189000.00,0.054054,74,0.0,4.0,0.054054,74,0.0,4,0.0,74,0.0,0.0,0.0,74,0.0,0.0,25.418919,74,33.0,33.0,0.000000,74,0,0,0.000000,74,0.0,0,0.000000,74,0.0,0
2,100013,2038692.0,96,2038692.0,2038692,-48.5,96,-48.5,-1,18159.919219,96,0.0,161420.22,131718.750000,96,157500,157500,5953.125000,96,0.0,157500.0,5953.125000,96,0.0,157500.0,0.0,96,0.0,0.0,0.0,96,0.0,0.0,1348.479375,96,0.0,7875.0,7168.346250,96,274.320,...,18101.079844,96,0.0,161420.22,18101.079844,96,0.0,161420.22,0.239583,96,0.0,7.0,0.239583,96,0.0,7,0.0,96,0.0,0.0,0.0,96,0.0,0.0,17.354167,96,22.0,22.0,0.000000,96,0,0,0.010417,96,0.0,1,0.010417,96,0.0,1
3,100021,2594025.0,17,2594025.0,2594025,-10.0,17,-10.0,-2,0.000000,17,0.0,0.00,675000.000000,17,675000,675000,0.000000,17,0.0,0.0,0.000000,17,0.0,0.0,0.0,17,0.0,0.0,0.0,17,0.0,0.0,0.000000,17,0.0,0.0,0.000000,17,0.000,...,0.000000,17,0.0,0.00,0.000000,17,0.0,0.00,0.000000,17,0.0,0.0,0.000000,17,0.0,0,0.0,17,0.0,0.0,0.0,17,0.0,0.0,0.000000,17,0.0,0.0,1.176471,17,2,2,0.000000,17,0.0,0,0.000000,17,0.0,0
4,100023,1499902.0,8,1499902.0,1499902,-7.5,8,-7.5,-4,0.000000,8,0.0,0.00,135000.000000,8,135000,225000,0.000000,8,0.0,0.0,0.000000,8,0.0,0.0,0.0,8,0.0,0.0,0.0,8,0.0,0.0,0.000000,8,0.0,0.0,0.000000,8,0.000,...,0.000000,8,0.0,0.00,0.000000,8,0.0,0.00,0.

In [13]:
pos_cash_balance_labels=[n+'_'+l for n in pos_cash_balance.columns.tolist() if n!='SK_ID_CURR' for l in ['mean','count','median','max']]
pos_cash_stats=pos_cash_balance.groupby('SK_ID_CURR').agg(['mean','count','median','max']).reset_index()
pos_cash_stats.columns=['SK_ID_CURR']+ pos_cash_balance_labels
pos_cash_stats.head()

,SK_ID_CURR,SK_ID_PREV_mean,SK_ID_PREV_count,SK_ID_PREV_median,SK_ID_PREV_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_count,MONTHS_BALANCE_median,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_count,CNT_INSTALMENT_median,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,CNT_INSTALMENT_FUTURE_count,CNT_INSTALMENT_FUTURE_median,CNT_INSTALMENT_FUTURE_max,NAME_CONTRACT_STATUS_mean,NAME_CONTRACT_STATUS_count,NAME_CONTRACT_STATUS_median,NAME_CONTRACT_STATUS_max,SK_DPD_mean,SK_DPD_count,SK_DPD_median,SK_DPD_max,SK_DPD_DEF_mean,SK_DPD_DEF_count,SK_DPD_DEF_median,SK_DPD_DEF_max
0,100001,1.584045e+06,9,1369693.0,1851984,-72.555556,9,-57.0,-53,4.000000,9,4.0,4.0,1.444444,9,1.0,4.0,0.888889,9,0.0,4,0.777778,9,0.0,7,0.777778,9,0.0,7
1,100002,1.038818e+06,19,1038818.0,1038818,-10.000000,19,-10.0,-1,24.000000,19,24.0,24.0,15.000000,19,15.0,24.0,0.000000,19,0.0,0,0.000000,19,0.0,0,0.000000,19,0.0,0
2,100003,2.297665e+06,28,2396755.0,2636178,-43.785714,28,-26.5,-18,10.107143,28,12.0,12.0,5.785714,28,6.0,12.0,0.285714,28,0.0,4,0.000000,28,0.0,0,0.000000,28,0.0,0
3,100004,1.564014e+06,4,1564014.0,1564014,-25.500000,4,-25.5,-24,3.750000,4,4.0,4.0,2.250000,4,2.5,4.0,1.000000,4,0.0,4,0.000000,4,0.0,0,0.000000,4,0.0,0
4,100005,2.495675e+06,11,2495675.0,2495675,-20.000000,11,-20.0,-15,10.636364,11,12.0,12.0,6.545455,11,7.0,12.0,1.000000,11,0.0,7,0.000000,11,0.0,0,0.000000,11,0.0,0


In [14]:
previous_applications_labels=[n+'_'+l for n in previous_applications.columns.tolist() if n!='SK_ID_CURR' for l in ['mean','count','median','max']]
previous_applications_stats=previous_applications.groupby('SK_ID_CURR').agg(['mean','count','median','max']).reset_index()
previous_applications_stats.columns=['SK_ID_CURR']+ previous_applications_labels
previous_applications_stats.head()

,SK_ID_CURR,SK_ID_PREV_mean,SK_ID_PREV_count,SK_ID_PREV_median,SK_ID_PREV_max,NAME_CONTRACT_TYPE_mean,NAME_CONTRACT_TYPE_count,NAME_CONTRACT_TYPE_median,NAME_CONTRACT_TYPE_max,AMT_ANNUITY_mean,AMT_ANNUITY_count,AMT_ANNUITY_median,AMT_ANNUITY_max,AMT_APPLICATION_mean,AMT_APPLICATION_count,AMT_APPLICATION_median,AMT_APPLICATION_max,AMT_CREDIT_mean,AMT_CREDIT_count,AMT_CREDIT_median,AMT_CREDIT_max,AMT_DOWN_PAYMENT_mean,AMT_DOWN_PAYMENT_count,AMT_DOWN_PAYMENT_median,AMT_DOWN_PAYMENT_max,AMT_GOODS_PRICE_mean,AMT_GOODS_PRICE_count,AMT_GOODS_PRICE_median,AMT_GOODS_PRICE_max,WEEKDAY_APPR_PROCESS_START_mean,WEEKDAY_APPR_PROCESS_START_count,WEEKDAY_APPR_PROCESS_START_median,WEEKDAY_APPR_PROCESS_START_max,HOUR_APPR_PROCESS_START_mean,HOUR_APPR_PROCESS_START_count,HOUR_APPR_PROCESS_START_median,HOUR_APPR_PROCESS_START_max,FLAG_LAST_APPL_PER_CONTRACT_mean,FLAG_LAST_APPL_PER_CONTRACT_count,FLAG_LAST_APPL_PER_CONTRACT_median,...,NAME_SELLER_INDUSTRY_mean,NAME_SELLER_INDUSTRY_count,NAME_SELLER_INDUSTRY_median,NAME_SELLER_INDUSTRY_max,CNT_PAYMENT_mean,CNT_PAYMENT_count,CNT_PAYMENT_median,CNT_PAYMENT_max,NAME_YIELD_GROUP_mean,NAME_YIELD_GROUP_count,NAME_YIELD_GROUP_median,NAME_YIELD_GROUP_max,PRODUCT_COMBINATION_mean,PRODUCT_COMBINATION_count,PRODUCT_COMBINATION_median,PRODUCT_COMBINATION_max,DAYS_FIRST_DRAWING_mean,DAYS_FIRST_DRAWING_count,DAYS_FIRST_DRAWING_median,DAYS_FIRST_DRAWING_max,DAYS_FIRST_DUE_mean,DAYS_FIRST_DUE_count,DAYS_FIRST_DUE_median,DAYS_FIRST_DUE_max,DAYS_LAST_DUE_1ST_VERSION_mean,DAYS_LAST_DUE_1ST_VERSION_count,DAYS_LAST_DUE_1ST_VERSION_median,DAYS_LAST_DUE_1ST_VERSION_max,DAYS_LAST_DUE_mean,DAYS_LAST_DUE_count,DAYS_LAST_DUE_median,DAYS_LAST_DUE_max,DAYS_TERMINATION_mean,DAYS_TERMINATION_count,DAYS_TERMINATION_median,DAYS_TERMINATION_max,NFLAG_INSURED_ON_APPROVAL_mean,NFLAG_INSURED_ON_APPROVAL_count,NFLAG_INSURED_ON_APPROVAL_median,NFLAG_INSURED_ON_APPROVAL_max
0,100001,1.369693e+06,1,1369693.0,1369693,1.000000,1,1.0,1,3951.000,1,3951.000,3951.000,24835.50,1,24835.50,24835.5,23787.00,1,23787.00,23787.0,2520.0,1,2520.0,2520.0,24835.50,1,24835.50,24835.5,0.000000,1,0.0,0,13.000000,1,13.0,13,1.0,1,1.0,...,2.000000,1,2.0,2,8.0,1,8.0,8.0,1.000000,1,1.0,1,13.0,1,13.0,13,365243.0,1,365243.0,365243.0,-1709.000000,1,-1709.0,-1709.0,-1499.000000,1,-1499.0,-1499.0,-1619.000000,1,-1619.0,-1619.0,-1612.000000,1,-1612.0,-1612.0,0.000000,1,0.0,0.0
1,100002,1.038818e+06,1,1038818.0,1038818,1.000000,1,1.0,1,9251.775,1,9251.775,9251.775,179055.00,1,179055.00,179055.0,179055.00,1,179055.00,179055.0,0.0,1,0.0,0.0,179055.00,1,179055.00,179055.0,2.000000,1,2.0,2,9.000000,1,9.0,9,1.0,1,1.0,...,0.000000,1,0.0,0,24.0,1,24.0,24.0,3.000000,1,3.0,3,15.0,1,15.0,15,365243.0,1,365243.0,365243.0,-565.000000,1,-565.0,-565.0,125.000000,1,125.0,125.0,-25.000000,1,-25.0,-25.0,-17.000000,1,-17.0,-17.0,0.000000,1,0.0,0.0
2,100003,2.281150e+06,3,2396755.0,2636178,0.666667,3,1.0,1,56553.990,3,64567.665,98356.995,435436.50,3,337500.00,900000.0,484191.00,3,348637.50,1035882.0,2295.0,3,0.0,6885.0,435436.50,3,337500.00,900000.0,1.666667,3,2.0,3,14.666667,3,15.0,17,1.0,3,1.0,...,6.333333,3,5.0,10,10.0,3,12.0,12.0,3.666667,3,4.0,4,9.0,3,9.0,11,365243.0,3,365243.0,365243.0,-1274.333333,3,-797.0,-716.0,-1004.333333,3,-647.0,-386.0,-1054.333333,3,-647.0,-536.0,-1047.333333,3,-639.0,-527.0,0.666667,3,1.0,1.0
3,100004,1.564014e+06,1,1564014.0,1564014,1.000000,1,1.0,1,5357.250,1,5357.250,5357.250,24282.00,1,24282.00,24282.0,20106.00,1,20106.00,20106.0,4860.0,1,4860.0,4860.0,24282.00,1,24282.00,24282.0,0.000000,1,0.0,0,5.000000,1,5.0,5,1.0,1,1.0,...,2.000000,1,2.0,2,4.0,1,4.0,4.0,4.000000,1,4.0,4,14.0,1,14.0,14,365243.0,1,365243.0,365243.0,-784.000000,1,-784.0,-784.0,-694.000000,1,-694.0,-694.0,-724.000000,1,-724.0,-724.0,-714.000000,1,-714.0,-714.0,0.000000,1,0.0,0.0
4,100005,2.176837e+06,2,2176837.0,2495675,0.500000,2,0.5,1,2406.600,2,2406.600,4813.200,22308.75,2,22308.75,44617.5,20076.75,2,20076.75,40153.5,2232.0,2,2232.0,4464.0,22308.75,2,22308.75,44617.5,2.000000,2,2.0,4,

In [15]:
train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,6,7,4,3,1,0.018801,-9461,-637,-3648.0,-2120,0.0,1,1,0,1,1,0,8,1.0,2,2,6,10,0,0,0,0,0,0,5,...,0.0205,0.0193,0.0000,0.00,3,0,0.0149,5,0,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,1,4,1,1,1,0.003541,-16765,-1188,-1186.0,-291,0.0,1,1,0,1,1,0,3,2.0,1,1,1,11,0,0,0,0,0,0,39,...,0.0787,0.0558,0.0039,0.01,3,0,0.0714,0,0,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,6,7,4,3,1,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,8,1.0,2,2,1,9,0,0,0,0,0,0,11,...,0.0000,0.0000,0.0000,0.00,0,1,0.0000,7,2,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,6,7,4,0,1,0.008019,-19005,-3039,-9833.0,-2437,0.0,1,1,0,1,0,0,8,2.0,2,2,6,17,0,0,0,0,0,0,5,...,0.0000,0.0000,0.0000,0.00,0,1,0.0000,7,2,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,6,7,4,3,1,0.028663,-19932,-3038,-4311.0,-3458,0.0,1,1,0,1,0,0,3,1.0,2,2,4,11,0,0,0,0,1,1,37,...,0.0000,0.0000,0.0000,0.00,0,1,0.0000,7,2,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
train_test_combo = (train.append(test).merge(bureau_stats, on = 'SK_ID_CURR', how = 'left')
                                       .merge(credit_card_stats, on = 'SK_ID_CURR', how = 'left')
                                        .merge(pos_cash_stats, on = 'SK_ID_CURR', how = 'left')
                                        .merge(previous_applications_stats, on = 'SK_ID_CURR', how = 'left'))

In [17]:
train_test_combo.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,NAME_SELLER_INDUSTRY_mean,NAME_SELLER_INDUSTRY_count,NAME_SELLER_INDUSTRY_median,NAME_SELLER_INDUSTRY_max,CNT_PAYMENT_mean,CNT_PAYMENT_count,CNT_PAYMENT_median,CNT_PAYMENT_max,NAME_YIELD_GROUP_mean,NAME_YIELD_GROUP_count,NAME_YIELD_GROUP_median,NAME_YIELD_GROUP_max,PRODUCT_COMBINATION_mean,PRODUCT_COMBINATION_count,PRODUCT_COMBINATION_median,PRODUCT_COMBINATION_max,DAYS_FIRST_DRAWING_mean,DAYS_FIRST_DRAWING_count,DAYS_FIRST_DRAWING_median,DAYS_FIRST_DRAWING_max,DAYS_FIRST_DUE_mean,DAYS_FIRST_DUE_count,DAYS_FIRST_DUE_median,DAYS_FIRST_DUE_max,DAYS_LAST_DUE_1ST_VERSION_mean,DAYS_LAST_DUE_1ST_VERSION_count,DAYS_LAST_DUE_1ST_VERSION_median,DAYS_LAST_DUE_1ST_VERSION_max,DAYS_LAST_DUE_mean,DAYS_LAST_DUE_count,DAYS_LAST_DUE_median,DAYS_LAST_DUE_max,DAYS_TERMINATION_mean,DAYS_TERMINATION_count,DAYS_TERMINATION_median,DAYS_TERMINATION_max,NFLAG_INSURED_ON_APPROVAL_mean,NFLAG_INSURED_ON_APPROVAL_count,NFLAG_INSURED_ON_APPROVAL_median,NFLAG_INSURED_ON_APPROVAL_max
0,100002,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,6,7,4,3,1,0.018801,-9461,-637,-3648.0,-2120,0.0,1,1,0,1,1,0,8,1.0,2,2,6,10,0,0,0,0,0,0,5,...,0.000000,1.0,0.0,0.0,24.000000,1.0,24.0,24.0,3.000000,1.0,3.0,3.0,15.000000,1.0,15.0,15.0,365243.000000,1.0,365243.0,365243.0,-565.000000,1.0,-565.0,-565.0,125.000000,1.0,125.0,125.0,-25.000000,1.0,-25.0,-25.0,-17.000000,1.0,-17.0,-17.0,0.000000,1.0,0.0,0.0
1,100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,1,4,1,1,1,0.003541,-16765,-1188,-1186.0,-291,0.0,1,1,0,1,1,0,3,2.0,1,1,1,11,0,0,0,0,0,0,39,...,6.333333,3.0,5.0,10.0,10.000000,3.0,12.0,12.0,3.666667,3.0,4.0,4.0,9.000000,3.0,9.0,11.0,365243.000000,3.0,365243.0,365243.0,-1274.333333,3.0,-797.0,-716.0,-1004.333333,3.0,-647.0,-386.0,-1054.333333,3.0,-647.0,-536.0,-1047.333333,3.0,-639.0,-527.0,0.666667,3.0,1.0,1.0
2,100004,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,6,7,4,3,1,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,8,1.0,2,2,1,9,0,0,0,0,0,0,11,...,2.000000,1.0,2.0,2.0,4.000000,1.0,4.0,4.0,4.000000,1.0,4.0,4.0,14.000000,1.0,14.0,14.0,365243.000000,1.0,365243.0,365243.0,-784.000000,1.0,-784.0,-784.0,-694.000000,1.0,-694.0,-694.0,-724.000000,1.0,-724.0,-724.0,-714.000000,1.0,-714.0,-714.0,0.000000,1.0,0.0,0.0
3,100006,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,6,7,4,0,1,0.008019,-19005,-3039,-9833.0,-2437,0.0,1,1,0,1,0,0,8,2.0,2,2,6,17,0,0,0,0,0,0,5,...,8.555556,9.0,10.0,10.0,15.333333,9.0,12.0,48.0,1.333333,9.0,1.0,4.0,5.000000,9.0,6.0,11.0,162330.222222,9.0,0.0,365243.0,40474.000000,9.0,0.0,365243.0,40704.000000,9.0,0.0,365243.0,81101.111111,9.0,0.0,365243.0,81103.000000,9.0,0.0,365243.0,0.000000,9.0,0.0,0.0
4,100007,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,6,7,4,3,1,0.028663,-19932,-3038,-4311.0,-3458,0.0,1,1,0,1,0,0,3,1.0,2,2,4,11,0,0,0,0,1,1,37,...,5.666667,6.0,4.0,10.0,20.666667,6.0,15.0,48.0,2.500000,6.0,2.5,4.0,8.166667,6.0,8.0,13.0,304369.166667,6.0,365243.0,365243.0,-1052.666667,6.0,-955.0,0.0,-697.666667,6.0,-535.0,346.0,60113.500000,6.0,-550.0,365243.0,60119.833333,6.0,-543.0,365243.0,0.500000,6.0,0.5,1.0


In [18]:
train_test_combo.drop(labels=['SK_ID_CURR'],axis=1,inplace=True)

In [19]:
train_df = train_test_combo.iloc[:train_shape,:]
test_df = train_test_combo.iloc[train_shape:,:]

In [20]:
param = {'objective' : 'binary',
          'boosting_type': 'gbdt',
          'metric' : 'auc',
          'nthread' : 4,
          'shrinkage_rate':0.025,
          'max_depth':8,
          'min_data_in_leaf':100,
          'min_child_weight': 2,
          'bagging_fraction':0.75,
          'feature_fraction':0.75,
          'min_split_gain':.01,
          'lambda_l1':1,
          'lambda_l2':1,
          'num_leaves':36}    





In [21]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(train_df, target, test_size=0.2, shuffle=True)

In [22]:
train_data=gbm.Dataset(train_x,label=train_y)
valid_data=gbm.Dataset(valid_x,label=valid_y)

In [23]:
lgbm = gbm.train(param,
                 train_data,
                 1000,
                 valid_sets=valid_data,
                 early_stopping_rounds= 100,
                 verbose_eval= 10
                 )

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.738094
[20]	valid_0's auc: 0.741916
[30]	valid_0's auc: 0.747875
[40]	valid_0's auc: 0.750353
[50]	valid_0's auc: 0.753273
[60]	valid_0's auc: 0.75557
[70]	valid_0's auc: 0.75802
[80]	valid_0's auc: 0.760295
[90]	valid_0's auc: 0.762678
[100]	valid_0's auc: 0.764456
[110]	valid_0's auc: 0.766258
[120]	valid_0's auc: 0.76807
[130]	valid_0's auc: 0.769487
[140]	valid_0's auc: 0.770794
[150]	valid_0's auc: 0.771931
[160]	valid_0's auc: 0.773044
[170]	valid_0's auc: 0.774426
[180]	valid_0's auc: 0.77509
[190]	valid_0's auc: 0.776003
[200]	valid_0's auc: 0.776829
[210]	valid_0's auc: 0.777726
[220]	valid_0's auc: 0.778449
[230]	valid_0's auc: 0.77905
[240]	valid_0's auc: 0.779641
[250]	valid_0's auc: 0.780132
[260]	valid_0's auc: 0.780487
[270]	valid_0's auc: 0.780852
[280]	valid_0's auc: 0.781314
[290]	valid_0's auc: 0.781736
[300]	valid_0's auc: 0.782167
[310]	valid_0's auc: 0.782578
[320]	valid_0's auc: 

In [24]:
from sklearn.metrics import roc_auc_score
train_y_lgb = lgbm.predict(train_x)
val_y_lgb = lgbm.predict(valid_x)
print("ROC for Train ROC" , roc_auc_score(train_y,train_y_lgb))
print("ROC for Validation ROC",roc_auc_score(valid_y,val_y_lgb))


ROC for Train ROC 0.8701760865826053
ROC for Validation ROC 0.7883279183710917


In [25]:
predictions = lgbm.predict(test_df)
submission = pd.read_csv('../input/sample_submission.csv')


submission.TARGET = predictions

submission.to_csv('lgbm_submission.csv', index=False)